In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/DAAG/spam7.csv')

In [3]:
df.head()

,Unnamed: 0,crl.tot,dollar,bang,money,n000,make,yesno
0,1,278,0.000,0.778,0.00,0.00,0.00,y
1,2,1028,0.180,0.372,0.43,0.43,0.21,y
2,3,2259,0.184,0.276,0.06,1.16,0.06,y
3,4,191,0.000,0.137,0.00,0.00,0.00,y
4,5,191,0.000,0.135,0.00,0.00,0.00,y


In [4]:
df['spam'] = df['yesno'].apply(lambda x: 0 if x == 'n' else 1)

In [5]:
df['spam'].value_counts()

0    2788
1    1813
Name: spam, dtype: int64

In [6]:
df.columns = ['#', 'crl.tot', 'dollar', 'bang', 'money', 'n000', 'make', 'yesno', 'spam']
df = df.drop(['#', 'yesno'], axis=1)
df.head()

,crl.tot,dollar,bang,money,n000,make,spam
0,278,0.000,0.778,0.00,0.00,0.00,1
1,1028,0.180,0.372,0.43,0.43,0.21,1
2,2259,0.184,0.276,0.06,1.16,0.06,1
3,191,0.000,0.137,0.00,0.00,0.00,1
4,191,0.000,0.135,0.00,0.00,0.00,1


In [7]:
X = df[['crl.tot', 'dollar', 'bang', 'money', 'n000', 'make']]
Y = df['spam']

columns = list(X.columns)
for c1 in range(0, len(columns)):    
    for c2 in range(c1 + 1, len(columns)):
        col1 = columns[c1]
        col2 = columns[c2]
        X[col1 + '_' + col2] = X[col1] * X[col2]
        
X.head()

,crl.tot,dollar,bang,money,n000,make,crl.tot_dollar,crl.tot_bang,crl.tot_money,crl.tot_n000,...,dollar_bang,dollar_money,dollar_n000,dollar_make,bang_money,bang_n000,bang_make,money_n000,money_make,n000_make
0,278,0.000,0.778,0.00,0.00,0.00,0.000,216.284,0.00,0.00,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000
1,1028,0.180,0.372,0.43,0.43,0.21,185.040,382.416,442.04,442.04,...,0.066960,0.07740,0.07740,0.03780,0.15996,0.15996,0.07812,0.1849,0.0903,0.0903
2,2259,0.184,0.276,0.06,1.16,0.06,415.656,623.484,135.54,2620.44,...,0.050784,0.01104,0.21344,0.01104,0.01656,0.32016,0.01656,0.0696,0.0036,0.0696
3,191,0.000,0.137,0.00,0.00,0.00,0.000,26.167,0.00,0.00,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000
4,191,0.000,0.135,0.00,0.00,0.00,0.000,25.785,0.00,0.00,...,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.0000,0.0000


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

RANDOM_SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)

model = GradientBoostingClassifier(random_state=RANDOM_SEED,learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features=None)
model.fit(X_train, y_train)

model.score(X_test, y_test)

#y_pred = model.predict(X_test)

#score(y_test, y_pred)

0.8653637350705755

In [9]:
model.feature_importances_

array([2.97159411e-02, 1.40769896e-01, 4.25720650e-02, 6.09020185e-03,
       2.47773348e-02, 1.16642741e-03, 8.67921568e-03, 6.14877389e-01,
       4.02447573e-02, 1.36581568e-03, 6.53881797e-03, 6.05665814e-02,
       7.22153860e-03, 3.42199906e-03, 1.69411057e-03, 6.46902133e-03,
       1.17577385e-03, 1.96526554e-03, 2.65508758e-05, 0.00000000e+00,
       6.61296634e-04])

In [11]:
from sklearn.model_selection import GridSearchCV

RANDOM_SEED = 42
param_grid = {'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
model = GradientBoostingClassifier(random_state=RANDOM_SEED, learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features=None)
clf = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.8621064060803475

In [21]:
import math
import numpy as np
from sklearn.tree import DecisionTreeClassifier

def AdaBoost_scratch(X,y, M=10, learning_rate = 1):
    # инициалиазция служебных переменных
    N = len(y)
    estimator_list, y_predict_list, estimator_error_list, estimator_weight_list, sample_weight_list = [], [],[],[],[]

    # инициализация весов
    sample_weight = np.ones(N) / N
    sample_weight_list.append(sample_weight.copy())

    # цикл по длине М
    for m in range(M):   

        # обучим базовую модель и получим предсказание
        estimator = DecisionTreeClassifier(max_depth = 1, max_leaf_nodes=2)
        estimator.fit(X, y, sample_weight=sample_weight)
        y_predict = estimator.predict(X)

        # Маска для ошибок классификации
        incorrect = (y_predict != y)

        # Оцениваем ошибку
        estimator_error = sum(sample_weight * incorrect) / sum(sample_weight)
        
        # Вычисляем вес нового алгоритма
        estimator_weight = learning_rate * math.log((1 - estimator_error) / estimator_error)

        # Получаем новые веса объектов
        sample_weight *= np.exp(estimator_weight * incorrect * ((sample_weight > 0) | (estimator_weight < 0)))

        # Сохраяем результаты данной итерации
        estimator_list.append(estimator)
        y_predict_list.append(list(y_predict))
        estimator_error_list.append(estimator_error)
        estimator_weight_list.append(estimator_weight)
        sample_weight_list.append(list(sample_weight))
        


    # Для удобства переведем в numpy.array   
    estimator_list = np.asarray(estimator_list)
    y_predict_list = np.asarray(y_predict_list)
    estimator_error_list = np.asarray(estimator_error_list)
    estimator_weight_list = np.asarray(estimator_weight_list)
    sample_weight_list = np.asarray(sample_weight_list)

    # Получим предсказания
    preds = (np.array([np.sign((y_predict_list[:,point] * estimator_weight_list).sum()) for point in range(N)]))
    print('Accuracy = ', (preds == y).sum() / N) 
    
    return estimator_list, estimator_weight_list, sample_weight_list

estimator_list, estimator_weight_list, sample_weight_list  = AdaBoost_scratch(X, Y, M=10, learning_rate=0.001)

Accuracy =  0.8228645946533363
